In [1]:
from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper

# Load environment variables from .env file
load_dotenv()


# Define Tools
def get_current_time(*args, **kwargs):
    """Returns the current time in H:MM AM/PM format."""
    import datetime

    now = datetime.datetime.now()
    return now.strftime("%I:%M %p")

wp_api_wrapper = WikipediaAPIWrapper(top_k_results=5)
ddg_api_wrapper = DuckDuckGoSearchAPIWrapper()

# Define the tools that the agent can use
tools = [
    Tool(
        name="Time",
        func=get_current_time,
        description="Useful for when you need to know the current time.",
    ),
    WikipediaQueryRun(api_wrapper=wp_api_wrapper),
    DuckDuckGoSearchRun(api_wrapper=ddg_api_wrapper)
]

# Load the correct JSON Chat Prompt from the hub
prompt = hub.pull("hwchase17/structured-chat-agent")

# Initialize a ChatOpenAI model
llm = ChatOpenAI(model="gpt-4o")

# Create a structured Chat Agent with Conversation Buffer Memory
# ConversationBufferMemory stores the conversation history, allowing the agent to maintain context across interactions
memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True)

# create_structured_chat_agent initializes a chat agent designed to interact using a structured prompt and tools
# It combines the language model (llm), tools, and prompt to create an interactive agent
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt)

# AgentExecutor is responsible for managing the interaction between the user input, the agent, and the tools
# It also handles memory to ensure context is maintained throughout the conversation
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory,  # Use the conversation memory to maintain context
    handle_parsing_errors=True,  # Handle any parsing errors gracefully
)

# Initial system message to set the context for the chat
# SystemMessage is used to define a message from the system to the agent, setting initial instructions or context
initial_message = "You are an AI assistant that can provide helpful answers using available tools.\nIf you are unable to answer, you can use the following tools: DuckDuckGo and Wikipedia."
memory.chat_memory.add_message(SystemMessage(content=initial_message))

# Chat Loop to interact with the user
while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        break

    # Add the user's message to the conversation memory
    memory.chat_memory.add_message(HumanMessage(content=user_input))

    # Invoke the agent with the user input and the current chat history
    response = agent_executor.invoke({"input": user_input})
    print("Bot:", response["output"])

    # Add the agent's response to the conversation memory
    memory.chat_memory.add_message(AIMessage(content=response["output"]))

c:\Users\phani\anaconda3\envs\rag\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


{
  "action": "duckduckgo_search",
  "action_input": "current US Election news"
}Former President Donald Trump and Vice President Kamala Harris are hitting the campaign trail Friday as they race to make their pitch to voters ahead of Election Day. Follow here for the latest ... Election Live Updates: Vance Makes Last Campaign Stops Before V.P. Debate. Senator JD Vance, Donald Trump's running mate, has a pair of events in the crucial battleground of Pennsylvania. Sept. 26. 47% Trump. 49% Harris. Nate Cohn Chief political analyst. In one sense, new Times/Siena polls in Arizona, Georgia and North Carolina are very clear: They are good numbers for Donald ... The latest political polls and polling averages from FiveThirtyEight. 24/7 Coverage of Breaking News and Live Events. UPDATED Sep. 27, 2024, at 11:35 AM. Latest Polls. ... An updating average of 2020 presidential general election polls, accounting for each poll's quality, sample size and recency. KEY. The latest news and updates on the

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


{
  "action": "Time",
  "action_input": {}
}

ToolException: Too many arguments to single-input tool Time.
                Consider using StructuredTool instead. Args: []

In [2]:
from langchain_community.tools import DuckDuckGoSearchResults

In [3]:
search = DuckDuckGoSearchResults()

In [4]:
search.run("Olympics 2024 medal tally")

'[snippet: Paris 2024 president Tony Estanguet unveiled the Olympic and Paralympic medals for the Games in February 2024, which on the obverse featured embedded hexagon-shaped tokens of scrap iron that had been taken from the original construction of the Eiffel Tower, with the Games logo engraved into it. Approximately 5,084 medals would be produced by the French mint Monnaie de Paris, and were designed ..., title: 2024 Summer Olympics medal table - Wikipedia, link: https://en.wikipedia.org/wiki/2024_Summer_Olympics_medal_table], [snippet: Olympic medal count for the 2024 Summer Games with updates. By Zachary Pereles. Jul 27, 2024 at 2:17 pm ET • 1 min read Getty Images. The 2024 Paris Olympics are underway after competition ..., title: 2024 Paris Olympics medal count: Tracker for how many gold, silver ..., link: https://www.cbssports.com/olympics/news/2024-paris-olympics-medal-count-tracker-for-how-many-gold-silver-bronze-medals-usa-each-country-has-won/], [snippet: CNN is tracking th